In [ ]:
import json
import os
import numpy as np

from typing import Any, Dict, Optional, List, Union
from dataclasses import dataclass, field

from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient

In [ ]:
from typing import Any, Dict, List, Optional
import json
import os
from langchain.chat_models import ChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient


class MCPManager():
    def __init__(self, config_file:str, llm_type:str):

        # 读取 MCP server 的配置文件
        self.config = self._load_config(config_file)

        # 初始化大模型
        self.llm, self.visual_llm = self._init_llm(llm_type)

        # MCP客户端和工具
        self.client: Optional[MultiServerMCPClient] = None
        self.tools: List = []
        self.tools_by_server: Dict[str, List] = {}

        # 对话历史
        self.conversation_history: List[Dict[str, str]] = []
        
        print("MCP管理器初始化完成")

    def _load_config(self, config_file: str) -> Dict[str, Any]:
        """加载配置文件"""
        try:
            with open(config_file, 'r', encoding='utf-8') as f:
                config = json.load(f)
            print(f"配置文件加载成功: {config_file}")
            return config
        except FileNotFoundError:
            print(f"⚠️ 配置文件未找到: {config_file}，使用默认配置")
            return {"servers": {}, "agent_permissions": {}}
        except json.JSONDecodeError as e:
            print(f"❌ 配置文件格式错误: {e}")

    def _init_llm(self) -> ChatOpenAI:
        """初始化大模型 - 从环境变量加载配置"""
        # 大模型配置只从环境变量加载
        llm_api_key = os.getenv("CHAT_LLM_API_KEY", "your_api_key_here")
        visual_llm_api_key = os.getenv("VISUAL_LLM_API_KEY", "your_api_key_here")
        base_url = os.getenv("LLM_BASE_URL", "https://dashscope.aliyuncs.com/compatible-mode/v1")
        chat_model_name = os.getenv("LLM_MODEL", "qwen3-max")
        visual_model_name = os.getenv("LLM_MODEL", "qwen-vl-max-latest")
        temperature = float(os.getenv("LLM_TEMPERATURE", "0.1"))
        max_tokens = int(os.getenv("LLM_MAX_TOKENS", "None"))

        print(f"[LLM INIT] Loaded from env -> chat_LLM_MODEL={chat_model_name}, visual_LLM_MODEL={visual_model_name}, LLM_TEMPERATURE={temperature}, LLM_MAX_TOKENS={max_tokens}, LLM_BASE_URL={base_url}")
        
        llm = ChatOpenAI(
            model=chat_model_name,
            api_key=llm_api_key,
            base_url=base_url,
            temperature=temperature,
            max_tokens=max_tokens
        )

        visual_llm = ChatOpenAI(
            model=visual_model_name,
            api_key=visual_llm_api_key,
            base_url=base_url,
            temperature=temperature,
            max_tokens=max_tokens
        )

        try:
            print(
                f"[LLM INIT] ChatOpenAI config -> model={getattr(llm, 'model', getattr(llm, 'model_name', None))}, "
                f"temperature={getattr(llm, 'temperature', None)}, max_tokens={getattr(llm, 'max_tokens', None)}"
            )
        except Exception as _:
            pass
        
        print(f"大模型初始化完成: {chat_model_name}, {visual_model_name} @ {base_url}")
        return llm, visual_llm

In [ ]:

class AgentState(MessagesState):
    """智能体状态管理 - 所有智能体间传递的核心状态"""
    # 原始输入
    image_path: Optional[str] = None
    crop_path: Optional[str] = None

    # 使用 field() 提供更灵活的默认值
    axis_info: Optional[dict] = field(default=None)
    OCR_detected_ticks: Optional[List[Dict[str, Union[float, int, List[int]]]]] = field(default=None)
    tick_pixel_raw: List[Dict[str, Union[int, float, None]]] = field(default=None)
    chart_border: Optional[dict] = field(default=None)
    tick_pixel_remap: List[Dict[str, Union[int, float, None]]] = field(default=None)
    pixel_to_value: Optional[dict] = field(default=None)
    curve_points: Optional[List[list[int]]] = field(default=None)
    curve_gray_values: Union[List[float], np.ndarray] = field(default=None)
    spectrum: Dict[str, Union[List[float], float]] = field(default=None)



    
    
    # 分析师报告（智能体间传递的核心数据）
    company_overview_report: str = ""  # 公司概述分析师报告
    market_report: str = ""  # 市场分析师报告
    sentiment_report: str = ""  # 情绪分析师报告
    news_report: str = ""  # 新闻分析师报告
    fundamentals_report: str = ""  # 基本面分析师报告
    shareholder_report: str = ""  # 股东结构分析师报告
    product_report: str = ""  # 产品分析师报告
    
    # 研究员辩论状态
    investment_debate_state: Dict[str, Any] = {}
    investment_plan: str = ""  # 研究经理决策
    
    # 交易员决策
    trader_investment_plan: str = ""  # 交易员投资计划
    
    # 风险管理辩论状态
    risk_debate_state: Dict[str, Any] = {}
    final_trade_decision: str = ""  # 最终交易决策
    
    # MCP工具调用历史
    mcp_tool_calls: List[Dict[str, Any]] = []
    
    # 智能体执行历史
    agent_execution_history: List[Dict[str, Any]] = []
    
    # 错误和警告信息
    errors: List[str] = []
    warnings: List[str] = []
    
    def add_agent_execution(self, agent_name: str, action: str, result: str, mcp_used: bool = False):
        """添加智能体执行记录"""
        from datetime import datetime
        self.agent_execution_history.append({
            "agent_name": agent_name,
            "action": action,
            "result": result,
            "mcp_used": mcp_used,
            "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        })
    
    def add_mcp_tool_call(self, agent_name: str, tool_name: str, tool_args: Dict, tool_result: Any):
        """添加MCP工具调用记录"""
        from datetime import datetime
        self.mcp_tool_calls.append({
            "agent_name": agent_name,
            "tool_name": tool_name,
            "tool_args": tool_args,
            "tool_result": str(tool_result),
            "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        })
    
    def add_error(self, error_msg: str):
        """添加错误信息"""
        self.errors.append(error_msg)
    
    def add_warning(self, warning_msg: str):
        """添加警告信息"""
        self.warnings.append(warning_msg)
    
    def get_all_reports(self) -> Dict[str, str]:
        """获取所有分析师报告"""
        return {
            "company_overview_report": self.company_overview_report,
            "market_report": self.market_report,
            "sentiment_report": self.sentiment_report,
            "news_report": self.news_report,
            "fundamentals_report": self.fundamentals_report,
            "shareholder_report": self.shareholder_report,
            "product_report": self.product_report
        }
    
    def get_debate_summary(self) -> str:
        """获取辩论摘要"""
        investment_history = self.investment_debate_state.get("history", "")
        risk_history = self.risk_debate_state.get("history", "")
        
        summary = ""
        if investment_history:
            summary += f"投资辩论历史:\n{investment_history}\n\n"
        if risk_history:
            summary += f"风险管理辩论历史:\n{risk_history}\n\n"
        
        return summary.strip()


In [ ]:
import json
import os
import uuid
import time
from datetime import datetime
from typing import Dict, Any, Optional


class ProgressTracker:
    """简化的进度跟踪器 - 输出核心agent结果并保存到JSON"""
    
    def __init__(self, session_id: str = None):
        # 生成强唯一的会话ID：微秒 + UUID短码，避免并发同秒冲突
        self.session_id = session_id or f"{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}_{uuid.uuid4().hex[:8]}"
        self.current_stage = ""
        self.current_agent = ""
        
        # 初始化dump文件夹和JSON文件
        self.dump_dir = os.path.join(os.path.dirname(__file__), "dump")
        os.makedirs(self.dump_dir, exist_ok=True)
        self.json_file = os.path.join(self.dump_dir, f"session_{self.session_id}.json")
        
        # 初始化JSON数据结构
        self.session_data = {
            "session_id": self.session_id,
            "created_at": datetime.now().isoformat(),
            "updated_at": datetime.now().isoformat(),
            "status": "active",
            "user_query": "",
            "active_agents": [],
            "stages": [],
            "agents": [],
            "actions": [],
            "mcp_calls": [],
            "errors": [],
            "warnings": [],
            "final_results": {}
        }
        
        # 首次写入时确保原子创建，若意外存在则重新生成ID
        self._init_json_file()
        print(f"🚀 会话开始: {self.session_id}")
    
    def _init_json_file(self):
        """原子创建JSON文件，避免并发命名冲突。"""
        try:
            # 尝试独占创建；如已存在则换一个ID
            while True:
                try:
                    with open(self.json_file, 'x', encoding='utf-8') as f:
                        json.dump(self.session_data, f, ensure_ascii=False, indent=2)
                    break
                except FileExistsError:
                    # 极小概率碰撞，重生成ID与路径
                    self.session_id = f"{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}_{uuid.uuid4().hex[:8]}"
                    self.json_file = os.path.join(self.dump_dir, f"session_{self.session_id}.json")
        except Exception as e:
            print(f"❌ 初始化JSON失败: {e}")

    def _save_json(self):
        """保存数据到JSON文件（Windows友好：带重试的原子替换，必要时回退为直接写）。"""
        self.session_data["updated_at"] = datetime.now().isoformat()
        tmp_path = f"{self.json_file}.{uuid.uuid4().hex}.tmp"
        try:
            with open(tmp_path, 'w', encoding='utf-8') as f:
                json.dump(self.session_data, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"❌ 写临时文件失败: {e}")
            return

        # Windows 下 os.replace 可能因目的文件被读取而临时拒绝访问；重试几次
        replaced = False
        for i in range(6):  # ~ 累计约 1.5 秒
            try:
                os.replace(tmp_path, self.json_file)
                replaced = True
                break
            except PermissionError as e:
                # 回退等待后重试
                time.sleep(0.25 * (i + 1))
            except Exception as e:
                print(f"❌ 替换JSON失败: {e}")
                break

        if not replaced:
            # 回退：直接覆盖写入目标（可能不是全原子，但尽量保证成功）
            try:
                with open(self.json_file, 'w', encoding='utf-8') as f:
                    json.dump(self.session_data, f, ensure_ascii=False, indent=2)
            except Exception as e:
                print(f"❌ 覆盖写入JSON失败: {e}")
            finally:
                # 清理残留临时文件
                try:
                    if os.path.exists(tmp_path):
                        os.remove(tmp_path)
                except Exception:
                    pass
    
    def update_user_query(self, query: str):
        """更新用户查询"""
        self.session_data["user_query"] = query
        self._save_json()
        print(f"📝 用户查询: {query}")

    def set_active_agents(self, active_agents):
        """记录本轮启用的智能体列表"""
        try:
            self.session_data["active_agents"] = list(active_agents or [])
            self._save_json()
        except Exception:
            pass
    
    def start_stage(self, stage_name: str, description: str = ""):
        """开始新阶段"""
        self.current_stage = stage_name
        stage_data = {
            "stage_name": stage_name,
            "description": description,
            "start_time": datetime.now().isoformat()
        }
        self.session_data["stages"].append(stage_data)
        self._save_json()
        print(f"📍 阶段开始: {stage_name}")
        if description:
            print(f"   描述: {description}")
    
    def start_agent(self, agent_name: str, action: str = "", system_prompt: str = "", user_prompt: str = "", context: str = ""):
        """开始智能体工作"""
        self.current_agent = agent_name
        agent_data = {
            "agent_name": agent_name,
            "action": action,
            "start_time": datetime.now().isoformat(),
            "status": "running",
            "result": "",
            "system_prompt": system_prompt,
            "user_prompt": user_prompt, 
            "context": context
        }
        self.session_data["agents"].append(agent_data)
        self._save_json()
        print(f"🤖 智能体开始工作: {agent_name}")
        if action:
            print(f"   执行: {action}")
    
    def complete_agent(self, agent_name: str, result: str = "", success: bool = True):
        """完成智能体工作"""
        # 更新对应的agent记录
        for agent in self.session_data["agents"]:
            if agent["agent_name"] == agent_name and agent["status"] == "running":
                agent["status"] = "completed" if success else "failed"
                agent["result"] = result
                agent["end_time"] = datetime.now().isoformat()
                break
        
        self._save_json()
        status = "✅ 成功" if success else "❌ 失败"
        print(f"🏁 智能体完成: {agent_name} - {status}")
        
        # 输出完整的agent结果内容
        if result:
            print(f"\n📋 {agent_name} 输出结果:")
            print("=" * 50)
            print(result)
            print("=" * 50)
    
    def add_agent_action(self, agent_name: str, action: str, details: Dict[str, Any] = None):
        """添加智能体行动记录"""
        action_data = {
            "agent_name": agent_name,
            "action": action,
            "details": details or {},
            "timestamp": datetime.now().isoformat()
        }
        self.session_data["actions"].append(action_data)
        self._save_json()
        print(f"🔄 {agent_name}: {action}")
    
    def add_mcp_tool_call(self, agent_name: str, tool_name: str, tool_args: Dict, tool_result: Any):
        """记录MCP工具调用"""
        mcp_data = {
            "agent_name": agent_name,
            "tool_name": tool_name,
            "tool_args": tool_args,
            "tool_result": str(tool_result),
            "timestamp": datetime.now().isoformat()
        }
        self.session_data["mcp_calls"].append(mcp_data)
        self._save_json()
        print(f"🔧 {agent_name} 调用工具: {tool_name}")
    
    def update_global_state(self, state_key: str, state_value: Any):
        """更新全局状态"""
        pass  # 简化：不再保存状态
    
    def update_debate_state(self, debate_type: str, debate_data: Dict[str, Any]):
        """更新辩论状态"""
        print(f"🗣️ 辩论更新: {debate_type} - 轮次 {debate_data.get('count', 0)}")
    
    def add_error(self, error_msg: str, agent_name: str = None):
        """添加错误记录"""
        error_data = {
            "error_msg": error_msg,
            "agent_name": agent_name or "",
            "timestamp": datetime.now().isoformat()
        }
        self.session_data["errors"].append(error_data)
        self._save_json()
        if agent_name:
            print(f"❌ {agent_name} 错误: {error_msg}")
        else:
            print(f"❌ 错误: {error_msg}")
    
    def add_warning(self, warning_msg: str, agent_name: str = None):
        """添加警告记录"""
        warning_data = {
            "warning_msg": warning_msg,
            "agent_name": agent_name or "",
            "timestamp": datetime.now().isoformat()
        }
        self.session_data["warnings"].append(warning_data)
        self._save_json()
        if agent_name:
            print(f"⚠️ {agent_name} 警告: {warning_msg}")
        else:
            print(f"⚠️ 警告: {warning_msg}")
    
    def set_final_results(self, results: Dict[str, Any]):
        """设置最终结果"""
        self.session_data["final_results"] = results
        self.session_data["status"] = "completed"
        self._save_json()
        print(f"🏁 会话完成")
        print("\n📊 最终结果:")
        print("=" * 60)
        for key, value in results.items():
            print(f"{key}: {value}")
        print("=" * 60)
    
    def log_workflow_start(self, workflow_info: Dict[str, Any]):
        """记录工作流开始"""
        print(f"🚀 工作流开始: {workflow_info.get('user_query', '')}")
    
    def log_workflow_completion(self, completion_info: Dict[str, Any]):
        """记录工作流完成"""
        status = "成功" if completion_info.get("success", False) else "失败"
        print(f"🏁 工作流完成: {status}")
    
    def log_agent_start(self, agent_name: str, context: Dict[str, Any] = None):
        """记录智能体开始工作"""
        self.start_agent(agent_name, context.get("action", "") if context else "")
    
    def log_agent_complete(self, agent_name: str, result: Any = None, context: Dict[str, Any] = None):
        """记录智能体完成工作"""
        result_str = str(result) if result else ""
        success = context.get("success", True) if context else True
        self.complete_agent(agent_name, result_str, success)
    
    def log_llm_call(self, agent_name: str, prompt_preview: str, context: Dict[str, Any] = None):
        """记录LLM调用"""
        self.add_agent_action(agent_name, "LLM调用")
    
    def log_error(self, agent_name: str, error: str, context: Dict[str, Any] = None):
        """记录错误"""
        self.add_error(error, agent_name)
    
    def get_session_summary(self) -> Dict[str, Any]:
        """获取会话摘要"""
        return {"session_id": self.session_id}

In [ ]:
import os
from abc import ABC, abstractmethod
from typing import Dict, Any, List, Optional
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage
import base64

# from .agent_states import AgentState
# from .mcp_manager import MCPManager

class BaseAgent(ABC):
    """基础智能体类 - 所有智能体的基类"""
    
    def __init__(self, agent_name: str, mcp_manager: MCPManager, state: AgentState):
        self.agent_name = agent_name
        self.mcp_manager = mcp_manager
        
        # 获取LLM实例
        self.llm = mcp_manager.llm
        
        # 不在初始化时获取工具，而是在使用时动态获取
        self.available_tools = []
        
        # 延迟创建智能体实例，等到MCP工具初始化完成后再创建
        self.agent = None

        self.base64 = self.image_to_base64(state.image_path)

        print(f"智能体 {agent_name} 初始化完成，MCP工具: {'启用' if self.mcp_enabled else '禁用'}")

    def image_to_base64(self, image_path):
        with open(image_path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
         
    async def call_llm_with_context(self, state, analysis_request, progress_tracker, with_image: bool = False):

        system_prompt = self.get_system_prompt()

        if self.agent_name.endswith("_debate"):
            context_prompt = state.context_prompt

            # 将系统和上下文组合成一个系统消息
            system_level_prompt = f"""{system_prompt}

        {context_prompt}"""
        else:
            system_level_prompt = system_prompt

        try:
            if with_image:
                analysis_content = [
                    {"type": "text", "text": analysis_request}
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            else:
                analysis_content = [{"type": "text", "text": analysis_request}]

            # 构建简单的消息列表，让框架自动处理工具绑定
            messages = [
                {"role": "system", "content": system_level_prompt},
                {"role": "user", "content": analysis_content}
            ]

            response = await self.agent.ainvoke({
                "messages": messages
            })

            # 解析和显示工具调用信息
            messages = response.get("messages", [])
            tool_calls_found = []

            for msg in messages:
                # 检查是否是工具调用消息
                if hasattr(msg, 'tool_calls') and msg.tool_calls:
                    for tool_call in msg.tool_calls:
                        tool_name = tool_call.get('name', 'unknown')
                        tool_args = tool_call.get('args', {})
                        tool_id = tool_call.get('id', 'unknown')
                        
                        print(f"🔧 [{self.agent_name}] 调用工具: {tool_name}")
                        print(f"   参数: {tool_args}")
                        
                        tool_calls_found.append({
                            'tool_name': tool_name,
                            'tool_args': tool_args,
                            'tool_id': tool_id
                        })
                # 检查是否是工具返回结果消息
                elif hasattr(msg, 'tool_call_id'):
                    tool_result = getattr(msg, 'content', 'No result')
                    print(f"📋 [{self.agent_name}] 工具返回结果: {str(tool_result)[:200]}...")
                    
                    # 找到对应的工具调用并记录完整信息
                    for tool_call in tool_calls_found:
                        if tool_call.get('tool_id') == getattr(msg, 'tool_call_id', None):
                            # 记录到progress_tracker
                            if progress_tracker:
                                progress_tracker.add_mcp_tool_call(
                                    agent_name=self.agent_name,
                                    tool_name=tool_call['tool_name'],
                                    tool_args=tool_call['tool_args'],
                                    tool_result=tool_result
                                )
                        
                        # # 记录到state
                        # if isinstance(state, dict):
                        #     if 'mcp_tool_calls' not in state:
                        #         state['mcp_tool_calls'] = []
                        #     state['mcp_tool_calls'].append({
                        #         'agent_name': self.agent_name,
                        #         'tool_name': tool_call['tool_name'],
                        #         'tool_args': tool_call['tool_args'],
                        #         'tool_result': str(tool_result),
                        #         'timestamp': datetime.now().isoformat()
                        #     })
                        # else:
                        #     state.add_mcp_tool_call(
                        #         agent_name=self.agent_name,
                        #         tool_name=tool_call['tool_name'],
                        #         tool_args=tool_call['tool_args'],
                        #         tool_result=tool_result
                        #     )
                
                        # 提取最终回复
                        if messages:
                            # 通常最后一个消息是最终的AI回复
                            final_message = messages[-1]
                            if hasattr(final_message, 'content'):
                                result = final_message.content
                            else:
                                result = "(无法提取内容)"
                        else:
                            result = "(未收到消息)"
                        
        except Exception as mcp_error:
            print(f"⚠️ [{self.agent_name}] MCP工具调用失败，回退到无工具模式: {mcp_error}")
            # 回退到无工具模式
            full_prompt = f"""{system_level_prompt}\n\n用户请求: {user_message}"""
            response = await self.llm.ainvoke([HumanMessage(content=full_prompt)])
            result = response.content
            else:
                # 如果没有启用MCP工具，直接调用LLM
                print(f"⚡ [{self.agent_name}] 正在调用LLM（无工具）...")
                full_prompt = f"""{system_level_prompt}\n\n用户请求: {user_message}"""
                response = await self.llm.ainvoke([HumanMessage(content=full_prompt)])
                result = response.content



In [ ]:
"""
astro_agents.py
天文光谱分析多代理系统定义
"""

from .base_agent import BaseAgent
from .mcp_manager import MCPManager
from .agent_states import AgentState
from .utils import user_query
from datetime import datetime

current_datetime = datetime.now()


# ---------------------------------------------------------
# 1. Visual Assistant — 负责图像理解与坐标阅读
# ---------------------------------------------------------
class SpectralVisualInterpreter(BaseAgent):
    """公司概述分析师 - 负责获取公司基础信息和概览"""
    
    def __init__(self, mcp_manager: MCPManager):
        super().__init__(
            agent_name="Spectral_Visual_Interpreter",
            mcp_manager=mcp_manager,
        )
    
    def get_system_prompt(self) -> str:
        return f"""
你是一位专业的【天文学光谱视觉分析师】，负责为后续分析师提供光谱图的基础信息。

工作原则：
- 按规则精确描述坐标轴
- 不对光谱进行数值计算，仅作定性判断
- 不解释光谱物理成因，只描述观测现象
- 如果图像不完整或分辨率低，请在回答中指出不确定性

请务必仔细完成任务，为后续分析打好基础。

"""
    
    async def ReadAxis(self, state: AgentState, progress_tracker=None) -> AgentState:
        """使用视觉模型读取坐标轴"""
        
        try:
            # 构建分析请求
            analysis_request = """
本次分析要求如下：

- 如果输入中不包含图像，请输出 “未输入图像”

- 如果输入中包含图像，请分析这张图的 x 轴和 y 轴。

- 严格按照以下 JSON Schema 输出分析结果，不要包含任何额外文本、解释或 Markdown 代码块：

{
  "x_axis": {
    "label and Unit": "str",
    "tick_range": {
      "min": float 或 int,
      "max": float 或 int
    },
    "ticks": [数值列表，按升序排列，包含 min 和 max]
  },
  "y_axis": {
    "label and Unit": "str",
    "tick_range": {
      "min": float 或 int,
      "max": float 或 int
    },
    "ticks": [数值列表，按升序排列，包含 min 和 max]
  }
}
"""

            # 调用LLM进行分析
            analysis_result = await self.call_llm_with_context(state, analysis_request, progress_tracker, with_image=True)
            
            # 格式化并保存结果
            formatted_result = self.format_output(analysis_result, state)
            if isinstance(state, dict):
                state['company_overview_report'] = formatted_result
                # 同时保存为company_details占位符供其他分析师使用
                state['company_details'] = analysis_result  # 保存原始内容，不包含格式化头部
            else:
                state.company_overview_report = formatted_result
                state.company_details = analysis_result
            
            print("✅ 公司概述信息收集完成")
            
        except Exception as e:
            error_msg = f"公司概述分析失败: {str(e)}"
            print(f"❌ {error_msg}")
            if isinstance(state, dict):
                if 'errors' not in state:
                    state['errors'] = []
                state['errors'].append(error_msg)
                state['company_overview_report'] = f"公司概述分析出现错误: {error_msg}"
                state['company_details'] = f"公司信息获取失败: {error_msg}"
            else:
                state.add_error(error_msg)
                state.company_overview_report = f"公司概述分析出现错误: {error_msg}"
                state.company_details = f"公司信息获取失败: {error_msg}"
        
        return state